In [2]:
import pandas as pd
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

seed = 42

In [10]:
BP_train = pd.read_pickle("dataset/BP_train_solution.pkl")
BP_test = pd.read_pickle("dataset/BP_test_solution.pkl")
SC_train = pd.read_pickle("dataset/SC_train_solution.pkl")
# SC_test = pd.read_pickle("dataset/SC_test_solution.pkl")
SC_test = pd.DataFrame(columns=BP_test.columns)

BP_train = BP_train.sample(frac=1, random_state=seed)
print(len(BP_train))
BP_test = BP_test.sample(frac=1, random_state=seed)
SC_train = SC_train.sample(frac=1, random_state=seed)
SC_test = SC_test.sample(frac=1, random_state=seed)

train = pd.concat([BP_train, SC_train], ignore_index=True)
test = pd.concat([BP_test, SC_test], ignore_index=True)

X_train = train.drop(columns=['score'])
Y_train = train['score']

X_test = test.drop(columns=['score'])
Y_test = test['score']

28287


/var/folders/hk/khnvb8jj7f5czvgthvjj_dlw0000gn/T/ipykernel_52981/3744045235.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  test = pd.concat([BP_test, SC_test], ignore_index=True)


In [9]:
params = {
    "n_estimators": [20, 50, 100],
    "min_samples_leaf": [5, 10, 20, 30],
    "max_depth": [5, 10, 20],
}

et = ExtraTreesRegressor(random_state=seed)
# gs = GridSearchCV(et, params)

# gs.fit(X_train, Y_train)
# print(sorted(gs.cv_results_))

# et.fit(X_train, Y_train)

# y_pred = gs.predict(X_test)
# df = pd.DataFrame({'y_test': Y_test, 'y_pred': y_pred})
scores = cross_val_score(et, X_train, Y_train, cv=5, scoring='r2')
print("R^2 scores:", scores)

R^2 scores: [0.999999   0.99999937 0.99815599 0.9997777  0.99977901]
